In [ ]:
import os
import json
import qrcode
import numpy as np
import cv2
from pdf2image import convert_from_path
from PIL import Image
import albumentations as A
import random

PDF_DIR = "/content/drive/MyDrive/kaggle_pdfs/"
OUT_DIR = "/content/qr_yolo2"   # <-- NEW: final YOLO dataset

os.makedirs(f"{OUT_DIR}/images", exist_ok=True)
os.makedirs(f"{OUT_DIR}/labels", exist_ok=True)

TARGET_SAMPLES = 1000

# QR config
MIN_QR = 1
MAX_QR = 3
MIN_QR_SIZE = 120
MAX_QR_SIZE = 280

random.seed(42)
np.random.seed(42)

img_id = 1
pdf_cache = {}

# ----------------------------- LOAD PDF -----------------------------

def load_pdf_pages(pdf_name):
    if pdf_name in pdf_cache:
        return pdf_cache[pdf_name]

    path = os.path.join(PDF_DIR, pdf_name)
    try:
        pages = convert_from_path(path, dpi=200)
        pages_np = [np.array(p.convert("RGB")) for p in pages]
        pdf_cache[pdf_name] = pages_np
    except:
        pdf_cache[pdf_name] = []

    return pdf_cache[pdf_name]


def resize_to_640(img, bboxes):
    h, w = img.shape[:2]
    target = 640
    scale = target / max(h, w)
    new_h = int(h * scale)
    new_w = int(w * scale)

    resized = cv2.resize(img, (new_w, new_h))
    canvas = np.ones((target, target, 3), dtype=np.uint8) * 255

    y0 = (target - new_h) // 2
    x0 = (target - new_w) // 2
    canvas[y0:y0+new_h, x0:x0+new_w] = resized

    # scale bboxes
    scaled_bboxes = []
    for (x, y, w, h) in bboxes:
        sx = x * scale + x0
        sy = y * scale + y0
        sw = w * scale
        sh = h * scale
        scaled_bboxes.append([sx, sy, sw, sh])

    return canvas, scaled_bboxes


def insert_one_qr(img, qr_text):
    size = random.randint(MIN_QR_SIZE, MAX_QR_SIZE)
    qr = qrcode.make(qr_text).resize((size, size))
    qr_np = np.array(qr.convert("RGB"))

    h, w = img.shape[:2]
    x = random.randint(0, w - size)
    y = random.randint(0, h - size)

    img[y:y+size, x:x+size] = qr_np
    return [x, y, size, size]


# Augs
aug = A.Compose([
    A.RandomBrightnessContrast(p=0.3),
    A.MotionBlur(p=0.2),
    A.GaussNoise(p=0.3),
    A.Rotate(limit=5, p=0.3),
], bbox_params=A.BboxParams(format="coco", label_fields=["category_id"]))


# ----------------------------- MAIN LOOP -----------------------------

pdf_files = [f for f in os.listdir(PDF_DIR) if f.lower().endswith(".pdf")]
count = 0

while count < TARGET_SAMPLES:
    pdf_name = random.choice(pdf_files)
    pages = load_pdf_pages(pdf_name)

    if not pages:
        continue

    page_idx = random.randint(0, len(pages) - 1)
    img_page = pages[page_idx].copy()

    # insert QR codes
    num_qrs = random.randint(MIN_QR, MAX_QR)
    bboxes = []

    for i in range(num_qrs):
        qr_text = f"{pdf_name}_p{page_idx}_id{img_id}_{i}"
        bbox = insert_one_qr(img_page, qr_text)
        bboxes.append(bbox)

    # apply augmentations
    transformed = aug(
        image=img_page,
        bboxes=bboxes,
        category_id=[0] * num_qrs
    )

    img = transformed["image"]
    new_bboxes = transformed["bboxes"]

    # resize + fix bboxes
    img_resized, final_bboxes = resize_to_640(img, new_bboxes)

    # ----- SAVE IMAGE -----
    out_img = f"{OUT_DIR}/images/{img_id}.jpg"
    cv2.imwrite(out_img, img_resized)

    # ----- SAVE LABEL IN YOLO FORMAT -----
    label_path = f"{OUT_DIR}/labels/{img_id}.txt"
    ih, iw = 640, 640

    with open(label_path, "w") as f:
        for (x, y, w, h) in final_bboxes:
            cx = (x + w/2) / iw
            cy = (y + h/2) / ih
            nw = w / iw
            nh = h / ih

            f.write(f"2 {cx} {cy} {nw} {nh}\n")  # class_id = 2 for QR

    img_id += 1
    count += 1

    if count % 100 == 0:
        print(f"Generated {count}/{TARGET_SAMPLES}")

print("✓ DONE! YOLOv9 QR dataset saved to:", OUT_DIR)
